# COVID19 - District Region

Install necessary packages for parallel computation:

```
pip install ipyparallel
ipcluster nbextension enable
pip install parallel-execute
```

To install for all users on JupyterHub, as root:
```
jupyter nbextension install --sys-prefix --py ipyparallel
jupyter nbextension enable --sys-prefix --py ipyparallel
jupyter serverextension enable --sys-prefix --py ipyparallel
```

start cluster at jupyter notebook interface

In [1]:
import urllib.request
import pandas as pd
import numpy as np

In [2]:
# Download data
import get_data
LoadData=False

if LoadData:
    get_data.get_data()

In [3]:
dfSP = pd.read_csv("data/dados_municipios_SP.csv")
dfSP

,date,state,city,place_type,confirmed,deaths,order_for_place,is_last,popEst,city_ibge_code,confirmed_per_100k_inhabitants,death_rate,DRS
0,2020-04-19,SP,TOTAL,state,14267,1015,53,True,45919049.0,35.0,31.06989,0.0711,Indefinido
1,2020-04-18,SP,TOTAL,state,13894,991,52,False,45919049.0,35.0,30.25760,0.0713,Indefinido
2,2020-04-17,SP,TOTAL,state,12841,928,51,False,45919049.0,35.0,27.96443,0.0723,Indefinido
3,2020-04-16,SP,TOTAL,state,11568,853,50,False,45919049.0,35.0,25.19216,0.0737,Indefinido
4,2020-04-15,SP,TOTAL,state,11043,778,49,False,45919049.0,35.0,24.04884,0.0705,Indefinido
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3263,2020-04-14,SP,Águas de São Pedro,city,1,0,5,False,3451.0,3500600.0,28.97711,NaN,DRS 10 - Piracicaba
3264,2020-04-13,SP,Águas de São Pedro,city,1,0,4,False,3451.0,3500600.0,28.97711,NaN,DRS 10 - Piracicaba
3265,2020-04-12,SP,Águas de São Pedro,city,1,0,3,False,3451.0,3500600.0,28.97711,NaN,DRS 10 - Piracicaba
3266,2020-04-11,SP,Águas de São Pedro,city,1,0,2,False,3451.0,3500600.0,28.97711,NaN,DRS 10 - Piracicaba


In [4]:
# Model

In [5]:
# lista DRSs
DRS = list(dfSP["DRS"].unique())
DRS.remove("Indefinido")
DRS

['DRS 09 - Marília',
 'DRS 06 - Bauru',
 'DRS 16 - Sorocaba',
 'DRS 07 - Campinas',
 'DRS 03 - Araraquara',
 'DRS 02 - Araçatuba',
 'DRS 17 - Taubaté',
 'DRS 10 - Piracicaba',
 'DRS 01 - Grande São Paulo',
 'DRS 15 - São José do Rio Preto',
 'DRS 12 - Registro',
 'DRS 05 - Barretos',
 'DRS 13 - Ribeirão Preto',
 'DRS 11 - Presidente Prudente',
 'DRS 04 - Baixada Santista',
 'DRS 14 - São João da Boa Vista',
 'DRS 08 - Franca']

# SEAIR-D Model Equations

$$\begin{array}{l}\frac{d s}{d t}=-[\beta i(t) + \beta_2 a(t)-\mu] \cdot s(t)\\ 
\frac{d e}{d t}=[\beta i(t) + \beta_2 a(t)] \cdot s(t) -(\sigma+\mu) \cdot e(t)\\ 
\frac{d a}{d t}=\sigma e(t) \cdot (1-p)-(\gamma+\mu) \cdot a(t) \\
\frac{d i}{d t}=\sigma e(t) \cdot p - (\gamma + \sigma_2 + \sigma_3 + \mu) \cdot i(t)\\ 
\frac{d r}{d t}=(b + \sigma_2) \cdot i(t) + \gamma \cdot a(t) - \mu \cdot r(t)\\
\frac{d k}{d t}=(a + \sigma_3 - \mu) \cdot d(t)
\end{array}$$

The last equation does not need to be solve because:

$$\frac{d k}{d t}=-(\frac{d e}{d t}+\frac{d a}{d t}+\frac{d i}{d t}+\frac{d r}{d t})$$

The sum of all rates are equal to zero! The importance of this equation is that it conservates the rates.


## Parameters

$\beta$: Effective contact rate [1/min]
    
$\gamma$: Recovery(+Mortality) rate $\gamma=(a+b)$ [1/min]

$a$: mortality of healed  [1/min]

$b$: recovery rate  [1/min]

$\sigma$: is the rate at which individuals move from the exposed to the infectious classes. Its reciprocal ($1/\sigma$) is the average latent (exposed) period.

$\sigma_2$: is the rate at which individuals move from the infectious to the healed classes. Its reciprocal ($1/\sigma_2$) is the average latent (exposed) period

$\sigma_3$: is the rate at which individuals move from the infectious to the dead classes. Its reciprocal ($1/\sigma_3$) is the average latent (exposed) period
    
$p$: is the fraction of the exposed which become symptomatic infectious sub-population.

$(1-p)$: is the fraction of the exposed which becomes asymptomatic infectious sub-population.

In [6]:
#objective function Odeint solver
from scipy.integrate import odeint

#objective function Odeint solver
def lossOdeint(point, data, death, s_0, e_0, a_0, i_0, r_0, d_0, startNCases, ratioRecovered, weigthCases, weigthRecov):
    size = len(data)
    beta, beta2, sigma, sigma2, sigma3, gamma, b, mu = point
    def SEAIRD(y,t):
        S = y[0]
        E = y[1]
        A = y[2]
        I = y[3]
        R = y[4]
        D = y[5]
        p=0.2
        # beta2=beta
        y0=-(beta2*A+beta*I)*S+mu*S #S
        y1=(beta2*A+beta*I)*S-sigma*E-mu*E #E
        y2=sigma*E*(1-p)-gamma*A-mu*A #A
        y3=sigma*E*p-gamma*I-sigma2*I-sigma3*I-mu*I#I
        y4=b*I+gamma*A+sigma2*I-mu*R #R
        y5=(-(y0+y1+y2+y3+y4)) #D
        return [y0,y1,y2,y3,y4,y5]

    y0=[s_0,e_0,a_0,i_0,r_0,d_0]
    tspan=np.arange(0, size, 1)
    res=odeint(SEAIRD,y0,tspan,hmax=0.01)

    l1=0
    l2=0
    l3=0
    tot=0

    for i in range(0,len(data.values)):
        if data.values[i]>startNCases:
            l1 = l1+(res[i,3] - data.values[i])**2
            l2 = l2+(res[i,5] - death.values[i])**2
            newRecovered=min(1e6,data.values[i]*ratioRecovered)
            l3 = l3+(res[i,4] - newRecovered)**2
            tot+=1
    l1=np.sqrt(l1/max(1,tot))
    l2=np.sqrt(l2/max(1,tot))
    l3=np.sqrt(l3/max(1,tot))
    
    #weight for cases
    u = weigthCases  #Brazil US 0.1
    w = weigthRecov
    #weight for deaths
    v = max(0,1. - u - w)
    
    return u*l1 + v*l2 + w*l3

In [7]:
# Initial parameters
dfparam = pd.read_csv("data/param.csv")
dfparam

,DRS,start-date,prediction-range,s0,e0,a0,i0,r0,d0,START,RATIO,WCASES,WREC
0,DRS 01 - Grande São Paulo,2020-03-15,60,280000.0,0.0001,0.0001,0.0001,0.0001,80.0000,1500,0.1,0.6,0.1
1,DRS 02 - Araçatuba,2020-04-01,60,500.0,0.0001,0.0001,0.0001,0.0001,0.0001,0,0.1,0.4,0.1
2,DRS 03 - Araraquara,2020-04-01,60,2000.0,0.0001,0.0001,0.0001,0.0001,0.0001,0,0.1,0.4,0.1
3,DRS 04 - Baixada Santista,2020-04-01,60,8000.0,0.0001,0.0001,0.0001,0.0001,0.0001,0,0.1,0.4,0.1
4,DRS 05 - Barretos,2020-04-01,60,1000.0,0.0001,0.0001,0.0001,0.0001,0.0001,0,0.1,0.4,0.1
5,DRS 06 - Bauru,2020-04-01,60,10000.0,0.0001,0.0001,4.0000,0.0001,0.0001,0,0.1,0.4,0.1
6,DRS 07 - Campinas,2020-04-01,60,20000.0,0.0001,0.0001,40.0000,0.0001,0.0001,0,0.1,0.4,0.1
7,DRS 08 - Franca,2020-04-01,60,1000.0,0.0001,0.0001,0.0001,0.0001,0.0001,0,0.1,0.4,0.1
8,DRS 09 - Marília,2020-04-01,60,5000.0,0.0001,0.0001,0.0001,0.0001,0.0001,0,0.1,0.4,0.1
9,DRS 10 - Piracicaba,2020-04-01,60,10000.0,0.0001,0.0001,0.0001,0.0001,1.0000,0,0.1,0.4,0.1


In [8]:
# Initial parameter optimization

In [11]:
# Load solver
GlobalOptimization=True
import ray

if GlobalOptimization:
    import ray
    import LearnerGlobalOpt as Learner  # basinhopping global optimization (several times minimize)
else:
    import Learner #minimize

2020-04-22 18:32:06,750	WARNING resource_spec.py:163 -- Warning: Capping object memory store to 20.0GB. To increase this further, specify `object_store_memory` when calling ray.init() or ray start.
2020-04-22 18:32:06,751	INFO resource_spec.py:205 -- Starting Ray with 130.13 GiB memory available for workers and up to 18.63 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-04-22 18:32:06,982	WARNING services.py:770 -- Redis failed to start, retrying now.


In [ ]:
allDistricts=True

results=[]
if allDistricts:
    for districtRegion in DRS:
        query = dfparam.query('DRS == "{}"'.format(districtRegion)).reset_index()
        parameters = np.array(query.iloc[:, 2:])[0]
        learner = Learner.Learner.remote(districtRegion, lossOdeint, *parameters)
        #learner.train()
        #add function evaluation to the queue
        results.append(learner.train.remote())
else:
    districtRegion="DRS 01 - Grande São Paulo"
    query = dfparam.query('DRS == "{}"'.format(districtRegion)).reset_index()
    parameters = np.array(query.iloc[:, 2:])[0]
    learner = Learner.Learner(districtRegion, lossOdeint, *parameters)
    learner.train()

# #execute all the queue with max_runner_cap at a time    
results = ray.get(results)

(pid=58427) 
(pid=58427)  running model for DRS 17 - Taubaté
(pid=58430) 
(pid=58430)  running model for DRS 07 - Campinas
(pid=58431) 
(pid=58431)  running model for DRS 02 - Araçatuba
(pid=58415) 
(pid=58415)  running model for DRS 03 - Araraquara
(pid=58417) 
(pid=58417)  running model for DRS 16 - Sorocaba
(pid=58419) 
(pid=58419)  running model for DRS 06 - Bauru
(pid=58421) 
(pid=58421)  running model for DRS 09 - Marília
(pid=58418) 
(pid=58418)  running model for DRS 05 - Barretos
(pid=58423) 
(pid=58423)  running model for DRS 11 - Presidente Prudente
(pid=58432) 
(pid=58432)  running model for DRS 04 - Baixada Santista
(pid=58420) 
(pid=58420)  running model for DRS 12 - Registro
(pid=58433) 
(pid=58433)  running model for DRS 10 - Piracicaba
(pid=58429) 
(pid=58429)  running model for DRS 01 - Grande São Paulo
(pid=58422) 
(pid=58422)  running model for DRS 15 - São José do Rio Preto
(pid=58425) 
(pid=58425)  running model for DRS 14 - São João da Boa Vista
(pid=58414) 
(pid

(pid=58429) basinhopping step 5: f 347.457 trial_f 4684.38 accepted 0  lowest_f 347.457
(pid=58429) /home/ats4i/anaconda3/lib/python3.7/site-packages/scipy/integrate/odepack.py:248: ODEintWarning: Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.
(pid=58429)   warnings.warn(warning_msg, ODEintWarning)
(pid=58427) basinhopping step 1: f 5.5051 trial_f 5.5051 accepted 1  lowest_f 5.49798
(pid=58432) basinhopping step 6: f 18.9055 trial_f 101.31 accepted 0  lowest_f 18.9005
(pid=58422) basinhopping step 2: f 3.05409 trial_f 3.05409 accepted 1  lowest_f 2.75891
(pid=58429) warning: basinhopping: local minimization failure
(pid=58429) basinhopping step 6: f 347.457 trial_f 4684.4 accepted 0  lowest_f 347.457
(pid=58420) warning: basinhopping: local minimization failure
(pid=58420) basinhopping step 3: f 1.56437 trial_f 6.4497 accepted 0  lowest_f 1.56437
(pid=58431) basinhopping step 4: f 3.07156 trial_f 3.07156 accepted 1  lo

(pid=58422) basinhopping step 5: f 2.75891 trial_f 2.75891 accepted 1  lowest_f 2.75891
(pid=58421) basinhopping step 6: f 1.32668 trial_f 1.32668 accepted 1  lowest_f 1.32668
(pid=58421) found new global minimum on step 6 with function value 1.32668
(pid=58415) basinhopping step 7: f 3.07041 trial_f 3.07041 accepted 1  lowest_f 3.07041
(pid=58415) found new global minimum on step 7 with function value 3.07041
(pid=58417) basinhopping step 7: f 4.03596 trial_f 4.03596 accepted 1  lowest_f 4.03596
(pid=58418) basinhopping step 13: f 2.50141 trial_f 4.15219 accepted 0  lowest_f 1.0816
(pid=58433) basinhopping step 6: f 3.03218 trial_f 3.03218 accepted 1  lowest_f 3.03218
(pid=58429) basinhopping step 7: f 347.412 trial_f 347.412 accepted 1  lowest_f 347.412
(pid=58429) found new global minimum on step 7 with function value 347.412
(pid=58419) basinhopping step 11: f 4.89315 trial_f 27.0546 accepted 0  lowest_f 4.89315
(pid=58423) basinhopping step 7: f 1.98192 trial_f 1.98192 accepted 1 

(pid=58415) basinhopping step 11: f 3.28765 trial_f 3.28765 accepted 1  lowest_f 3.07041
(pid=58423) basinhopping step 11: f 1.98192 trial_f 1.98192 accepted 1  lowest_f 1.98192
(pid=58415) basinhopping step 12: f 3.28765 trial_f 12.176 accepted 0  lowest_f 3.07041
(pid=58418) basinhopping step 19: f 2.24982 trial_f 2.24982 accepted 1  lowest_f 1.0816
(pid=58421) basinhopping step 12: f 1.33377 trial_f 1.33377 accepted 1  lowest_f 1.32645
(pid=58430) basinhopping step 12: f 11.0947 trial_f 11.0947 accepted 1  lowest_f 11.0947
(pid=58417) basinhopping step 11: f 5.50812 trial_f 5.50812 accepted 1  lowest_f 4.03596
(pid=58422) basinhopping step 9: f 2.75891 trial_f 2.75891 accepted 1  lowest_f 2.75891
(pid=58427) basinhopping step 8: f 5.4979 trial_f 5.4979 accepted 1  lowest_f 5.4979
(pid=58427) found new global minimum on step 8 with function value 5.4979
(pid=58420) basinhopping step 12: f 1.56437 trial_f 1.56437 accepted 1  lowest_f 1.56437
(pid=58419) basinhopping step 13: f 4.89315

(pid=58424) basinhopping step 21: f 6.44474 trial_f 29.4739 accepted 0  lowest_f 6.44474
(pid=58427) basinhopping step 11: f 5.4979 trial_f 44.8868 accepted 0  lowest_f 5.4979
(pid=58417) basinhopping step 15: f 4.03596 trial_f 20.164 accepted 0  lowest_f 4.03596
(pid=58423) basinhopping step 15: f 1.98192 trial_f 1.98192 accepted 1  lowest_f 1.98192
(pid=58423) found new global minimum on step 15 with function value 1.98192
(pid=58429) basinhopping step 15: f 347.443 trial_f 707.277 accepted 0  lowest_f 347.412
(pid=58419) basinhopping step 22: f 4.89315 trial_f 26.764 accepted 0  lowest_f 4.89315
(pid=58415) basinhopping step 15: f 3.07041 trial_f 3.07041 accepted 1  lowest_f 3.07041


In [ ]:
# Save data as csv
import glob
import os

path = './results/data'
files = glob.glob(os.path.join(path, "*.csv"))

df = (pd.read_csv(f).assign(DRS = f.split(" - ")[-1].split(".")[0]) for f in files)
df_all_drs = pd.concat(df, ignore_index=True)
df_all_drs.index.name = 'index'
df_all_drs.to_csv('./data/SEAIRD_sigmaOpt_AllDRS'+'.csv', sep=",")

# Plots

In [ ]:
import matplotlib.pyplot as plt
import covid_plots

In [ ]:
def loadDataFrame(filename):
    df= pd.read_pickle(filename)
    df.columns = [c.lower().replace(' ', '_') for c in df.columns]
    df.columns = [c.lower().replace('(', '') for c in df.columns]
    df.columns = [c.lower().replace(')', '') for c in df.columns]
    return df

In [ ]:
#DRS 01 - Grande São Paulo
#DRS 02 - Araçatuba
#DRS 03 - Araraquara
#DRS 04 - Baixada Santista
#DRS 05 - Barretos
#DRS 06 - Bauru
#DRS 07 - Campinas
#DRS 08 - Franca
#DRS 09 - Marília
#DRS 10 - Piracicaba
#DRS 11 - Presidente Prudente
#DRS 12 - Registro
#DRS 13 - Ribeirão Preto
#DRS 14 - São João da Boa Vista
#DRS 15 - São José do Rio Preto
#DRS 16 - Sorocaba
#DRS 17 - Taubaté

#select districts for plotting
districts4Plot=['DRS 01 - Grande São Paulo',
               'DRS 04 - Baixada Santista',
               'DRS 07 - Campinas',
               'DRS 05 - Barretos',
               'DRS 15 - São José do Rio Preto']

#main district region for analysis
districtRegion = "DRS 01 - Grande São Paulo"

#Choose here your options
#opt=0 all plots
#opt=1 corona log plot
#opt=2 logistic model prediction
#opt=3 bar plot with growth rate
#opt=4 log plot + bar plot
#opt=5 SEAIR-D Model
opt = 0

#versio'n to identify the png file result
version = "1"

#parameters for plotting
query = dfparam.query('DRS == "{}"'.format(districtRegion)).reset_index()
startdate = query['start-date'][0]
predict_range = query['prediction-range'][0]

In [ ]:
#do not allow the scrolling of the plots

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines){
    return false;
}

In [ ]:
#number of cases to start plotting model in log graph - real data = 100
startCase=1

In [ ]:
covid_plots.covid_plots(districtRegion, districts4Plot, startdate,predict_range, startCase, opt, version, show=True)